<a href="https://colab.research.google.com/github/katarinagresova/GearNet/blob/main/Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The goal of this notebook is to use GearNet ([Zhang, Zuobai, et al., 2023](https://doi.org/10.48550/arXiv.2203.06125)) for generating structural embedding of protein.

**Input:** protein structure in `.pdb` format  
**Output:** embedding vector with 3072 features

# Setup

## Install required libraries.

[Torchdrug](https://torchdrug.ai/) is build on top of PyTorch and tailored for drug discovery. GearNet is using Torchdrug.

Note: installing `torchdrug` takes several (up to 20) minutes.

In [ ]:
!pip install torch
!pip install torchdrug
!pip install easydict pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 12.7 MB/s eta 0:00:00
  Created wheel for torch-cluster: filename=torch_cluster-1.6.1-cp310-cp310-linux_x86_64.whl size=1974468 sha256=08941be1bd946d9972c410177bc3a0fb8907b236f6a1700db64a33fafd01a1a7
  Stored in directory: /root/.cache/pip/wheels/2e/ed/ac/1df43a8ff5b9bd2f44042636b98a60b2b4027d3ce8e8a3185f
  Created wheel for torch-scatter: filename=torch_scatter-2.1.1-cp3

## Download model and data

Pre-trained models weights for GearNet are stored [here](https://zenodo.org/record/7593637). Several weigths are available, optained using different training techniques.

For the data, I chose some example protein [Free fatty acid receptor 2](https://alphafold.ebi.ac.uk/entry/O15552).

Currently, GearNet works only with `.pdb` files. To load data, you use `data.Protein.from_pdb()` method. Unfortunately, there is no `data.Protein.from_mmcif()`. Under the hood, they are using [rdkit](https://www.rdkit.org/) to parse files, but adding support for parsing `mmcif` files is still [an open issue](https://github.com/rdkit/rdkit/issues/2054).

In [ ]:
!wget https://zenodo.org/record/7593637/files/mc_gearnet_edge.pth
!mkdir models
!mv mc_gearnet_edge.pth models/mc_gearnet_edge.pth

--2023-08-24 09:52:00--  https://zenodo.org/record/7593637/files/mc_gearnet_edge.pth
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80700937 (77M) [application/octet-stream]
Saving to: ‘mc_gearnet_edge.pth’

mc_gearnet_edge.pth 100%[===================>]  76.96M   608KB/s    in 2m 24s  

2023-08-24 09:54:25 (547 KB/s) - ‘mc_gearnet_edge.pth’ saved [80700937/80700937]



In [ ]:
!wget https://alphafold.ebi.ac.uk/files/AF-O15552-F1-model_v4.pdb
!mkdir data
!mv AF-O15552-F1-model_v4.pdb data/AF-O15552-F1-model_v4.pdb

--2023-08-24 09:54:26--  https://alphafold.ebi.ac.uk/files/AF-O15552-F1-model_v4.pdb
Resolving alphafold.ebi.ac.uk (alphafold.ebi.ac.uk)... 34.149.152.8
Connecting to alphafold.ebi.ac.uk (alphafold.ebi.ac.uk)|34.149.152.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘AF-O15552-F1-model_v4.pdb’

AF-O15552-F1-model_     [ <=>                ] 213.49K  --.-KB/s    in 0.002s  

2023-08-24 09:54:26 (111 MB/s) - ‘AF-O15552-F1-model_v4.pdb’ saved [218618]



# Prepare data

In [ ]:
from torchdrug import core, datasets, tasks, models, transforms, data, layers
from torchdrug.layers import geometry

In [ ]:
transform = transforms.ProteinView(view="residue")
graph_construction_model = layers.GraphConstruction(node_layers=[geometry.AlphaCarbonNode()],
                                                    edge_layers=[geometry.SpatialEdge(radius=10.0, min_distance=5),
                                                                 geometry.KNNEdge(k=10, min_distance=5),
                                                                 geometry.SequentialEdge(max_distance=2)],
                                                    edge_feature="gearnet")

In [ ]:
PROTEIN_PATH = './data/AF-O15552-F1-model_v4.pdb'
protein = data.Protein.from_pdb(PROTEIN_PATH, atom_feature="position", bond_feature="length", residue_feature="symbol")

with protein.residue():
    protein.residue_feature = protein.residue_feature.to_dense()

item = {"graph": protein}
item = transform(item)

_protein = data.Protein.pack([item['graph']])
protein = graph_construction_model(_protein)

# Prepare model

In [ ]:
import torch

WEIGHTS_PATH = './models/mc_gearnet_edge.pth'

# define model architecturemodel
gearnet_edge = models.GearNet(input_dim=21, hidden_dims=[512, 512, 512, 512, 512, 512],
                              num_relation=7, edge_input_dim=59, num_angle_bin=8,
                              batch_norm=True, concat_hidden=True, short_cut=True, readout="sum")

if torch.cuda.is_available():
  net = torch.load(WEIGHTS_PATH)
else:
  net = torch.load(WEIGHTS_PATH, map_location=torch.device('cpu'))

gearnet_edge.load_state_dict(net)
gearnet_edge.eval()

GeometryAwareRelationalGraphNeuralNetwork(
  (layers): ModuleList(
    (0): GeometricRelationalGraphConv(
      (batch_norm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (self_loop): Linear(in_features=21, out_features=512, bias=True)
      (linear): Linear(in_features=147, out_features=512, bias=True)
    )
    (1-5): 5 x GeometricRelationalGraphConv(
      (batch_norm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (self_loop): Linear(in_features=512, out_features=512, bias=True)
      (linear): Linear(in_features=3584, out_features=512, bias=True)
    )
  )
  (spatial_line_graph): SpatialLineGraph()
  (edge_layers): ModuleList(
    (0): GeometricRelationalGraphConv(
      (batch_norm): BatchNorm1d(21, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (self_loop): Linear(in_features=59, out_features=21, bias=True)
      (linear): Linear(in_features=472, out_features=21, bias=True)


# Compute embeddings

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gearnet_edge = gearnet_edge.to(device)
protein = protein.to(device)

In [ ]:
%timeit gearnet_edge(protein, protein.node_feature.float(), all_loss=None, metric=None)

114 ms ± 5.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
output = gearnet_edge(protein, protein.node_feature.float(), all_loss=None, metric=None)
output['graph_feature']

tensor([[ 7982.5234,   -62.6140,   -93.1467,  ..., 14019.3818,  3402.5784,
         12156.4658]], grad_fn=<ScatterAddBackward0>)

In [ ]:
import pandas as pd

pd.DataFrame(output['graph_feature'][0].detach().numpy()).describe()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


09:55:22   NumExpr defaulting to 2 threads.


,0
count,3072.000000
mean,1221.709106
std,4182.804688
min,-3180.315674
25%,-41.301127
50%,80.202488
75%,701.187408
max,91498.945312
